# **Installing Packages**

In [ ]:
!pip install tensorflow
!pip install keras

# **Importing Packages**

In [32]:
import pickle
import numpy as np
import pandas as pd
import sklearn.metrics
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import files
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Reading Data From Google Drive**

In [41]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv("/content/drive/My Drive/IntelliTech-DataSet/Features/EssaySet08_Features.csv", index_col =0)
df_train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0.1,ID,Essay,Rater_1 Score,Rater_2 Score,Rater_3 Score,Total Score,Trait 1,Trait 2,Trait 5,...,Semantic_Similarity,Connectivity_Index,Flesch_Reading_Ease_Score,Flesh_Kincaid_Grade,Dale_Challl_Readability_Score,Unique_Words_LD,Lexical_Diversity,Lexical_Density,Lexical_Originality,Lexical_Accuracy
0,0,20716,A long time ago when I was in third grade I h...,18,16,NaN,34,7,8,7,...,0.362023,26.879644,110.128739,5.124865,1.818356,198,0.237126,449,0.000391,17
1,1,20717,Softball has to be one of the single most gre...,21,26,46.0,46,5,5,5,...,0.335941,66.077254,85.292453,7.466403,6.508761,259,0.300116,532,0.000511,7
2,2,20718,"Some people like making people laugh, I love ...",15,20,40.0,40,4,4,4,...,0.344118,106.268978,86.015535,5.802192,6.368023,274,0.282766,572,0.000541,7
3,3,20719,"""LAUGHTER"" @CAPS1 I hang out with my friends...",12,20,30.0,30,3,3,3,...,0.251418,58.240182,91.707794,6.086874,5.850764,206,0.269281,442,0.000407,15
4,4,20721,Well ima tell a story about the time i got @CA...,11,15,NaN,26,6,5,5,...,0.366273,42.164157,100.450854,6.972390,1.997412,137,0.193503,436,0.000270,17


# **Extracting Features**

In [42]:
feats = ['Total Score', 'Sent_Count', 'Word_Count', 'Char_Count',
       'Avg_Word_Count', 'Avg_Sentence_Length', 'Syllable_Count', 'Difficult_Word_Count', 'Verb_Count', 'Noun_Count',
       'Adj_Count', 'Conj_Count', 'Adverb_Count', 'pNoun_Count',
       'Spelling_Mistakes_Count','Count_Fullstops', 'Count_Exclamation',
       'Count_Comma', 'Count_Questionmark', 'Count_Hyphens',
       'Capitalization_Errors', 'Grammar_Error_Count', 'Semantic_Similarity', 'Connectivity_Index',
       'Flesch_Reading_Ease_Score', 'Flesh_Kincaid_Grade',
       'Dale_Challl_Readability_Score', 'Unique_Words_LD', 'Lexical_Diversity',
       'Lexical_Density', 'Lexical_Originality', 'Lexical_Accuracy']
df_train = df_train.loc[:, df_train.columns.isin(feats)] 

In [ ]:
## Only for EssaySet04

#mask = df_train.isnull().any(axis=1)
#nan_rows = df_train[mask]
#print(nan_rows)
#df_train = df_train.dropna(

In [44]:
y_col = "Total Score"
X = df_train.drop(y_col, axis=1)
y = df_train[y_col]

In [45]:
df_train.head()

,Total Score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,Syllable_Count,Difficult_Word_Count,Verb_Count,Noun_Count,...,Semantic_Similarity,Connectivity_Index,Flesch_Reading_Ease_Score,Flesh_Kincaid_Grade,Dale_Challl_Readability_Score,Unique_Words_LD,Lexical_Diversity,Lexical_Density,Lexical_Originality,Lexical_Accuracy
0,34,39,835,3421,3.282635,21.410256,740,40,138,86,...,0.362023,26.879644,110.128739,5.124865,1.818356,198,0.237126,449,0.000391,17
1,46,30,863,3963,3.687138,28.766667,942,79,125,129,...,0.335941,66.077254,85.292453,7.466403,6.508761,259,0.300116,532,0.000511,7
2,40,39,969,4397,3.672859,24.846154,1095,92,132,159,...,0.344118,106.268978,86.015535,5.802192,6.368023,274,0.282766,572,0.000541,7
3,30,30,765,3415,3.521569,25.500000,807,46,100,112,...,0.251418,58.240182,91.707794,6.086874,5.850764,206,0.269281,442,0.000407,15
4,26,26,708,2943,3.247175,27.230769,659,29,117,66,...,0.366273,42.164157,100.450854,6.972390,1.997412,137,0.193503,436,0.000270,17


# **Splitting Data Into Training And Validation**

In [46]:
X = np.asarray(X).astype(np.float32)  #Converting datatypes to float because tensor accepts only float datatype.
#Error: Failed to convert a NumPy array to a Tensor (Unsupported object type int)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72018)

# **Model Training: Baseline NN**

In [17]:
X.dtypes

AttributeError: ignored

In [48]:
# The model architecture includes two Dense layers, with the first layer having 32 units and a sigmoid activation function, and the second layer having 1 unit 
# (since this is a regression task with a single target variable to predict).
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(Dense(1))

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_absolute_error"])

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print("Test loss:", score[0])
print("Test mean absolute error:", score[1])

Epoch 1/1000
16/16 [==============================] - 2s 27ms/step - loss: 1428.7866 - mean_absolute_error: 37.3185 - val_loss: 1368.6707 - val_mean_absolute_error: 36.6381
Epoch 2/1000
16/16 [==============================] - 0s 9ms/step - loss: 1390.5760 - mean_absolute_error: 36.8047 - val_loss: 1335.6917 - val_mean_absolute_error: 36.1856
Epoch 3/1000
16/16 [==============================] - 0s 11ms/step - loss: 1360.8242 - mean_absolute_error: 36.3994 - val_loss: 1313.6676 - val_mean_absolute_error: 35.8800
Epoch 4/1000
16/16 [==============================] - 0s 9ms/step - loss: 1338.7816 - mean_absolute_error: 36.0968 - val_loss: 1292.1592 - val_mean_absolute_error: 35.5790
Epoch 5/1000
16/16 [==============================] - 0s 8ms/step - loss: 1317.2080 - mean_absolute_error: 35.7956 - val_loss: 1270.8226 - val_mean_absolute_error: 35.2779
Epoch 6/1000
16/16 [==============================] - 0s 7ms/step - loss: 1295.8306 - mean_absolute_error: 35.4954 - val_loss: 1249.7972 -

The lower the test loss, the better the model is at making accurate predictions on new data. Typically, you would expect the training loss to decrease with each epoch, as the model learns to fit the training data more accurately. However, if the validation loss starts to increase while the training loss continues to decrease, this indicates that the model is overfitting and is not generalizing well to new data.

In [ ]:
print(sorted(sklearn.metrics.SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'top_k_accuracy', 'v_measure_score']


## **GridSearch**

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation="sigmoid"))   #RELU
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mean_absolute_error"])
    return model

model = KerasRegressor(build_fn=create_model, verbose=0)

param_grid = {
    "batch_size": [16, 32, 64],
    "epochs": [50, 100, 150]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)


<ipython-input-27-c89a1c7669d9>:13: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)


Best Parameters:  {'batch_size': 64, 'epochs': 150}


The negative score obtained from grid search means that the mean squared error (MSE) of the best model found is -0.7096900022811872, where the negative sign indicates that the mean squared error was negated to convert it to a score.To interpret this result, you can take the absolute value of the negative score to obtain the mean squared error (MSE) of the best model, which is approximately 0.709.

In [ ]:
print("Best Score: ", grid_search.best_score_)

Best Score:  -0.7299944392493802


## **Model Training: LSTM**  

In [30]:
# Creating Checkpoint to save the best model with minimum MAE 
checkpoint_filepath = './chkpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_mae',
    mode='min',
    save_best_only=True)

# Define the model architecture
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(14, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
adam = Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mae'])

# Train the model
model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=1000, batch_size=64, verbose=1, validation_split=0.2 , callbacks=[model_checkpoint_callback])

# Evaluate the model on the test set
score = model.evaluate(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test, verbose=0)
print('Test loss:', score[0])
print('Test MAE:', score[1])


Epoch 1/1000
15/16 [===========================>..] - ETA: 0s - loss: 1.7029 - mae: 1.0669

16/16 [==============================] - 22s 1s/step - loss: 1.6826 - mae: 1.0609 - val_loss: 0.8629 - val_mae: 0.8039
Epoch 2/1000
15/16 [===========================>..] - ETA: 0s - loss: 0.9348 - mae: 0.8064

16/16 [==============================] - 14s 954ms/step - loss: 0.9392 - mae: 0.8071 - val_loss: 0.8233 - val_mae: 0.7739
Epoch 3/1000
15/16 [===========================>..] - ETA: 0s - loss: 0.8589 - mae: 0.7781

16/16 [==============================] - 14s 898ms/step - loss: 0.8512 - mae: 0.7739 - val_loss: 0.7461 - val_mae: 0.7420
Epoch 4/1000
16/16 [==============================] - ETA: 0s - loss: 0.7609 - mae: 0.7205

16/16 [==============================] - 14s 961ms/step - loss: 0.7609 - mae: 0.7205 - val_loss: 0.5662 - val_mae: 0.6115
Epoch 5/1000
15/16 [===========================>..] - ETA: 0s - loss: 0.5168 - mae: 0.5690

...

16/16 [==============================] - 14s 934ms/step - loss: 0.3746 - mae: 0.4862 - val_loss: 0.3539 - val_mae: 0.4888
Epoch 36/1000
16/16 [==============================] - 1s 74ms/step - loss: 0.3689 - mae: 0.4850 - val_loss: 0.3608 - val_mae: 0.4915
Epoch 37/1000
16/16 [==============================] - 1s 73ms/step - loss: 0.3832 - mae: 0.4969 - val_loss: 0.4069 - val_mae: 0.5121
Epoch 38/1000
16/16 [==============================] - 1s 44ms/step - loss: 0.3850 - mae: 0.4908 - val_loss: 0.3570 - val_mae: 0.4932
Epoch 39/1000
16/16 [==============================] - 1s 45ms/step - loss: 0.3670 - mae: 0.4908 - val_loss: 0.3555 - val_mae: 0.4895
Epoch 40/1000
16/16 [==============================] - 1s 47ms/step - loss: 0.3673 - mae: 0.4822 - val_loss: 0.3796 - val_mae: 0.5005
Epoch 41/1000
16/16 [==============================] - 1s 42ms/step - loss: 0.3712 - mae: 0.4900 - val_loss: 0.3765 - val_mae: 0.4965
Epoch 42/1000
16/16 [==============================] - 1s 45ms/step - loss

16/16 [==============================] - 14s 897ms/step - loss: 0.3499 - mae: 0.4747 - val_loss: 0.3515 - val_mae: 0.4877
Epoch 57/1000
16/16 [==============================] - 1s 74ms/step - loss: 0.3601 - mae: 0.4817 - val_loss: 0.3512 - val_mae: 0.4906
Epoch 58/1000
16/16 [==============================] - ETA: 0s - loss: 0.3704 - mae: 0.4880

16/16 [==============================] - 14s 935ms/step - loss: 0.3704 - mae: 0.4880 - val_loss: 0.3500 - val_mae: 0.4866
Epoch 59/1000
16/16 [==============================] - 1s 45ms/step - loss: 0.3647 - mae: 0.4857 - val_loss: 0.3513 - val_mae: 0.4902
Epoch 60/1000
16/16 [==============================] - 1s 44ms/step - loss: 0.3685 - mae: 0.4921 - val_loss: 0.4002 - val_mae: 0.5102
Epoch 61/1000
16/16 [==============================] - 1s 47ms/step - loss: 0.3723 - mae: 0.4898 - val_loss: 0.3957 - val_mae: 0.5062
Epoch 62/1000
16/16 [==============================] - 1s 50ms/step - loss: 0.3618 - mae: 0.4849 - val_loss: 0.3824 - val_mae: 0.5020
Epoch 63/1000
16/16 [==============================] - 1s 45ms/step - loss: 0.3649 - mae: 0.4822 - val_loss: 0.3704 - val_mae: 0.4961
Epoch 64/1000
16/16 [==============================] - 1s 48ms/step - loss: 0.3530 - mae: 0.4772 - val_loss: 0.3534 - val_mae: 0.4889
Epoch 65/1000
16/16 [==============================] - 1s 49ms/step - loss

16/16 [==============================] - 13s 857ms/step - loss: 0.3631 - mae: 0.4786 - val_loss: 0.3502 - val_mae: 0.4866
Epoch 71/1000
16/16 [==============================] - 1s 48ms/step - loss: 0.3572 - mae: 0.4789 - val_loss: 0.3532 - val_mae: 0.4945
Epoch 72/1000
16/16 [==============================] - 1s 44ms/step - loss: 0.3635 - mae: 0.4883 - val_loss: 0.3505 - val_mae: 0.4872
Epoch 73/1000
16/16 [==============================] - 1s 43ms/step - loss: 0.3634 - mae: 0.4781 - val_loss: 0.3509 - val_mae: 0.4925
Epoch 74/1000
16/16 [==============================] - 1s 43ms/step - loss: 0.3574 - mae: 0.4844 - val_loss: 0.3506 - val_mae: 0.4871
Epoch 75/1000
16/16 [==============================] - 1s 71ms/step - loss: 0.3563 - mae: 0.4801 - val_loss: 0.3602 - val_mae: 0.4902
Epoch 76/1000
16/16 [==============================] - ETA: 0s - loss: 0.3575 - mae: 0.4769

16/16 [==============================] - 13s 892ms/step - loss: 0.3575 - mae: 0.4769 - val_loss: 0.3593 - val_mae: 0.4859
Epoch 77/1000
16/16 [==============================] - 1s 75ms/step - loss: 0.3575 - mae: 0.4792 - val_loss: 0.3533 - val_mae: 0.4927
Epoch 78/1000
16/16 [==============================] - 1s 76ms/step - loss: 0.3471 - mae: 0.4763 - val_loss: 0.3563 - val_mae: 0.4885
Epoch 79/1000
16/16 [==============================] - 1s 74ms/step - loss: 0.3515 - mae: 0.4784 - val_loss: 0.3529 - val_mae: 0.4896
Epoch 80/1000
16/16 [==============================] - 1s 72ms/step - loss: 0.3563 - mae: 0.4781 - val_loss: 0.3538 - val_mae: 0.4899
Epoch 81/1000
16/16 [==============================] - 1s 46ms/step - loss: 0.3555 - mae: 0.4845 - val_loss: 0.3951 - val_mae: 0.5085
Epoch 82/1000
16/16 [==============================] - 1s 42ms/step - loss: 0.3562 - mae: 0.4833 - val_loss: 0.3879 - val_mae: 0.5046
Epoch 83/1000
16/16 [==============================] - 1s 44ms/step - loss

In [31]:
#Fetching the best model saved using checkpoints
new_model = tf.keras.models.load_model("./chkpt")

#Testing data on the model saved 
score = new_model.evaluate(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test, verbose=0)

print('Test loss:', score[0])
print('Test MAE:', score[1])

Test loss: 0.37931984663009644
Test MAE: 0.484415203332901


Saving Model

In [ ]:
with open('model02_LSTM.pkl', 'wb') as f:
  pickle.dump(new_model, f)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_1
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-03-01 18:12:32        82680
metadata.json                                  2023-03-01 18:12:32           64
config.json                                    2023-03-01 18:12:32         4156


In [ ]:
files.download('/content/model02_LSTM.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>